# Quadratic Unconstrained Binary Optimization (QUBO) Solver

This Jupyter notebook provides a comprehensive approach to solving Quadratic Unconstrained Binary Optimization (QUBO) problems using multiple solvers. The notebook is structured to make it easy to understand and apply the different techniques to tackle QUBO problems.

## Notebook Structure

The notebook is divided into **three main sections**, each focusing on a different solver. In each section, you'll find:

- **Helper Methods**: Functions to assist in ingesting the QUBO matrix, preparing it for processing, and formatting the input for each solver.
- **Solver Implementation**: Detailed steps that explain how the specific solver operates on the given QUBO problem.
- **Results and Analysis**: The output of each solver is presented and analyzed to evaluate its performance on the QUBO instance.

## Key Highlights
- **Ingesting QUBO Matrix**: Each section includes methods to input a QUBO matrix, ensuring consistency in data handling.
- **Solver Comparison**: The notebook enables a comparative analysis of the solvers by standardizing the inputs and output format.
- **Versatile Solvers**: Different solving approaches are demonstrated, providing insights into the strengths and weaknesses of each method.

This setup will help you better understand the underlying mechanics of QUBO solvers and provide a robust framework for applying them to your optimization problems.


In [1]:
from amplify import FixstarsClient
from amplify import GurobiClient
from amplify import DWaveSamplerClient
from amplify import VariableGenerator
from amplify import solve
from datetime import timedelta
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

FIXSTAR_TOKEN = os.getenv('FIXSTAR_TOKEN')
DWAVE_TOKEN = os.getenv('DWAVE_TOKEN')

## Sample problem: Set partitioning

In [3]:
# Set partitioning

gen = VariableGenerator()
m = gen.matrix("Binary", 8)

q = m.quadratic
s = [25, 7, 13, 31, 42, 17, 21, 10]
total = sum(s)
print(total)

166


In [4]:
q = m.quadratic
for i in range(8):
    for j in range(8):
        q[i, j] = s[i] * (s[i] - total) if i == j else s[i] * s [j]
print(q)

[[-3525.   175.   325.   775.  1050.   425.   525.   250.]
 [  175. -1113.    91.   217.   294.   119.   147.    70.]
 [  325.    91. -1989.   403.   546.   221.   273.   130.]
 [  775.   217.   403. -4185.  1302.   527.   651.   310.]
 [ 1050.   294.   546.  1302. -5208.   714.   882.   420.]
 [  425.   119.   221.   527.   714. -2533.   357.   170.]
 [  525.   147.   273.   651.   882.   357. -3045.   210.]
 [  250.    70.   130.   310.   420.   170.   210. -1560.]]


In [5]:
model = m
print(model)

(x^T) Q x + (p^T) x + c
where:
  x = [q_0, q_1, q_2, q_3, q_4, q_5, q_6, q_7],
  Q = [[-3525.,   175.,   325.,   775.,  1050.,   425.,   525.,   250.],
       [  175., -1113.,    91.,   217.,   294.,   119.,   147.,    70.],
       [  325.,    91., -1989.,   403.,   546.,   221.,   273.,   130.],
       [  775.,   217.,   403., -4185.,  1302.,   527.,   651.,   310.],
       [ 1050.,   294.,   546.,  1302., -5208.,   714.,   882.,   420.],
       [  425.,   119.,   221.,   527.,   714., -2533.,   357.,   170.],
       [  525.,   147.,   273.,   651.,   882.,   357., -3045.,   210.],
       [  250.,    70.,   130.,   310.,   420.,   170.,   210., -1560.]],
  p = [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
  c = 0


## Sample problem: Max Cut

In [6]:
### Max cut

m2 = gen.matrix("Binary", 5)
q2 = m2.quadratic
q2[0, 0] = -2
q2[1, 1] = -2
q2[2, 2] = -3
q2[3, 3] = -3
q2[4, 4] = -2
q2[0, 1] = 1
q2[0, 2] = 1
q2[1, 0] = 1
q2[1, 3] = 1
q2[2, 0] = 1
q2[2, 3] = 1
q2[2, 4] = 1
q2[3, 1] = 1
q2[3, 2] = 1
q2[3, 4] = 1
q2[4, 2] = 1
q2[4, 3] = 1
print(q2)

[[-2.  1.  1.  0.  0.]
 [ 1. -2.  0.  1.  0.]
 [ 1.  0. -3.  1.  1.]
 [ 0.  1.  1. -3.  1.]
 [ 0.  0.  1.  1. -2.]]


In [7]:
model2 = m2
print(model2)

(x^T) Q x + (p^T) x + c
where:
  x = [q'_0, q'_1, q'_2, q'_3, q'_4],
  Q = [[-2.,  1.,  1.,  0.,  0.],
       [ 1., -2.,  0.,  1.,  0.],
       [ 1.,  0., -3.,  1.,  1.],
       [ 0.,  1.,  1., -3.,  1.],
       [ 0.,  0.,  1.,  1., -2.]],
  p = [ 0.,  0.,  0.,  0.,  0.],
  c = 0


## Fixstar Amplify

In [10]:
## Initialise Fixstars Client

clientFS = FixstarsClient()
clientFS.token = FIXSTAR_TOKEN 
clientFS.parameters.timeout = 1000 

In [11]:
## Solve set partitioning and view result
result = solve(model, clientFS)

print(result.best.objective)
print(result.best.values)

-6889.0
{q_0: 1, q_1: 0, q_2: 0, q_3: 1, q_4: 0, q_5: 1, q_6: 0, q_7: 1}


In [12]:
## Solve max cut and view result 
result2 = solve(model2, clientFS)

print(result2.best.objective)
print(result2.best.values)

-5.0
{q'_0: 1, q'_1: 0, q'_2: 0, q'_3: 1, q'_4: 1}


## Gurobi

In [ ]:
## Initialise Gurobi Client

clientG = GurobiClient()
clientG.library_path = "/Library/gurobi1103/macos_universal2/lib/libgurobi110.dylib"
clientG.parameters.time_limit = timedelta(seconds=100)

In [ ]:
## Solve set partitioning and view result Gurobi
resultG = solve(model, clientG)

print(resultG.best.objective)
print(resultG.best.values)

In [ ]:
## Solve max cut and view result Gurobi
resultG2 = solve(model2, clientG)

print(resultG2.best.objective)
print(resultG2.best.values)

## D-Wave

In [8]:
clientDWave = DWaveSamplerClient()
clientDWave.token = DWAVE_TOKEN
clientDWave.solver = "Advantage_system4.1"
clientDWave.parameters.num_reads = 1000

In [9]:
## Solve set partitioning and view result
result = solve(model, clientDWave)

print(result.best.objective)
print(result.best.values)

/Users/ck/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


-6889.0
{q_0: 1, q_1: 0, q_2: 0, q_3: 1, q_4: 0, q_5: 1, q_6: 0, q_7: 1}
